In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.functions import current_date, lit

In [0]:
df_consumos = (
    spark.read.table("production.raw.raw_tb_consumos")
    .select(
        F.col("consumo_id"),
        F.col("hospede_id"),
        F.col("quarto_id"),
        F.col("data_consumo"),
        F.col("tipo_servico"),
        F.col("descricao"),
        F.round(F.col("valor").cast("double"), 2).alias("valor"),  
        F.col("pago"),
    )
    .withColumn("data_consumo", F.to_date(F.col("data_consumo")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_consumos.limit(5))


In [0]:
df = spark.sql("""
               select * from production.raw.raw_tb_consumos
               """)
display(df.columns)

In [0]:
df_consumos = (
    spark.read.table("production.raw.raw_tb_consumos")
    .select(
        F.col("consumo_id"),
        F.col("hospede_id"),
        F.col("quarto_id"),
        F.col("data_consumo"),
        F.col("tipo_servico"),
        F.col("descricao"),
        F.round(F.col("valor").cast("double"), 2).alias("valor"),  
        F.col("pago"),

        consumo_id
        reserva_id
        hospede_id
        hotel_id
        quarto_id
        data_consumo
        tipo_servico
        descricao
        valor
        pago
    )
    .withColumn("data_consumo", F.to_date(F.col("data_consumo")))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast("date"))
)

display(df_consumos.limit(5))


In [0]:
catalog_table = "production.trusted.tb_faturas"

if spark.catalog.tableExists(catalog_table):
    delta_table = DeltaTable.forName(spark, catalog_table)
    
    # Merge (upsert) com atualização da update_date
    delta_table.alias("target").merge(
        df_faturas.alias("source"),
        "target.consumo_id = source.consumo_id"
    ).whenMatchedUpdate(
        set = {
            **{col: f"source.{col}" for col in df_faturas.columns if col != "update_date"},
            "update_date": "current_date()"
        }
    ).whenNotMatchedInsertAll() \
     .execute()
else:
    # Se não existir, cria a tabela Delta incluindo a update_date
    (
        df_faturas
        .withColumn("update_date", lit(""))
        .write
        .format("delta")
        .saveAsTable(catalog_table)
    )